In [ ]:
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

## Prediction ##

Let's work with Galton's dataset again.  As a reminder, he looks at a family, and for each child, he measured the average height of the child's two parents, and the height of the child when they reach adulthood.  The former is called the "midparent height" in this notebook.  (Why average height of the two parents?  Well, he was thinking that if the dad was tall or the mom was tall, then their child might be more likely to be tall; and if they were both tall, then that might have an even stronger effect, so the height of both parents is relevant.)  We're going to investigate how well one could predict the future height of a couple's kid, based on the height of the two parents.

In [ ]:
galton = Table.read_table('data/galton.csv')

In [ ]:
heights = Table().with_columns(
    'MidParent', galton.column('midparentHeight'),
    'Child', galton.column('childHeight')
    )

In [ ]:
heights

Recall that often a good thing to do anytime you have a new dataset is to visualize the data.  Let's look at a scatter plot to see if there seems to be any association between midparent height and eventual height of their child.

In [ ]:
heights.scatter('MidParent')

Looks like there might be a weak association.  Can we use this for prediction?  If we know the height of the two parents, can we predict how tall their kid will be when their kid grows up?  Let's work out how to do that.  The principle is to look for other couples who are similar (in terms of midparent height), where we know how tall their kid turned out to be, and use that to predict.  There might be multiple other couples that are similar, so we'll average the heights of all of their kids.

In [ ]:
def predict_child(h):
    """Return a prediction of the height of a child 
    whose parents have a midparent height of h.
    
    The prediction is the average height of the children 
    whose midparent height is in the range h plus or minus 0.5 inches.
    """
    
    close_points = heights.where('MidParent', are.between(h-0.5, h + 0.5))
    return np.mean(close_points.column('Child'))

In [ ]:
predict_child(72)

In [ ]:
heights_with_predictions = heights.with_column(
    'Prediction', heights.apply(predict_child, 'MidParent')
    )

We'll visualize the predictions against this dataset.  Blue dots are the observed data, yellow is the prediction we would have made (with this method).  You can see that the taller the parents are, the taller we predict the child will be.

In [ ]:
heights_with_predictions.scatter('MidParent')

## Association ##

Let's look at another example. This time we'll look at a dataset of different models of cars, with various attributes for each model of car.  We'll investigate what attributes have an association between them.

In [ ]:
hybrid = Table.read_table('data/hybrid.csv')

In [ ]:
hybrid

In [ ]:
hybrid.sort('msrp', descending=True)

Let's check whether there is an association between miles-per-gallon and the manufacturer's suggested retail price (msrp).

In [ ]:
hybrid.scatter('mpg', 'msrp')

Looks like there is some kind of association.  Is it a linear association?  Why do you think there is an association?  On first glance, one might have thought that building a car that gets better miles-per-gallon requires better technology, which would lead to a more expensive car -- but that's not what we're seeing.  Any guesses why?

How about acceleration vs msrp?  Is there an association?  A linear association?  Why?

In [ ]:
hybrid.scatter('acceleration', 'msrp')

In [ ]:
suv = hybrid.where('class', 'SUV')
suv.num_rows

In [ ]:
suv.scatter('acceleration', 'msrp')

In [ ]:
suv.scatter('mpg', 'msrp')

In [ ]:
def standard_units(x):
    "Convert any array of numbers to standard units."
    return (x - np.average(x)) / np.std(x)

In [ ]:
Table().with_columns(
    'mpg (standard units)',  standard_units(suv.column('mpg')), 
    'msrp (standard units)', standard_units(suv.column('msrp'))
).scatter(0, 1)
plots.xlim(-3, 3)
plots.ylim(-3, 3);

In [ ]:
suv.scatter('acceleration', 'msrp')

In [ ]:
Table().with_columns(
    'acceleration (standard units)', standard_units(suv.column('acceleration')), 
    'msrp (standard units)',         standard_units(suv.column('msrp'))
).scatter(0, 1)
plots.xlim(-3, 3)
plots.ylim(-3, 3);

## Correlation ##

In [ ]:
def r_scatter(r):
    plots.figure(figsize=(5,5))
    "Generate a scatter plot with a correlation approximately r"
    x = np.random.normal(0, 1, 1000)
    z = np.random.normal(0, 1, 1000)
    y = r*x + (np.sqrt(1-r**2))*z
    plots.scatter(x, y, color='darkblue', s=20)
    plots.xlim(-4, 4)
    plots.ylim(-4, 4)

In [ ]:
r_scatter(.5)

## Calculating $r$ ##

In [ ]:
x = np.arange(1, 7, 1)
y = make_array(2, 3, 1, 5, 2, 7)
t = Table().with_columns(
        'x', x,
        'y', y
    )
t

In [ ]:
t.scatter('x', 'y', s=30, color='red')

In [ ]:
t = t.with_columns(
        'x (standard units)', standard_units(x),
        'y (standard units)', standard_units(y)
    )
t

In [ ]:
t.scatter(2, 3)

In [ ]:
t = t.with_columns('product of standard units', t.column(2) * t.column(3))
t

In [ ]:
# r is the average of the products of the standard units

r = np.average(t.column(2) * t.column(3))
r

In [ ]:
def correlation(t, x, y):
    """t is a table; x and y are column labels"""
    x_in_standard_units = standard_units(t.column(x))
    y_in_standard_units = standard_units(t.column(y))
    return np.average(x_in_standard_units * y_in_standard_units)

In [ ]:
correlation(t, 'x', 'y')

In [ ]:
correlation(suv, 'mpg', 'msrp')

In [ ]:
correlation(suv, 'acceleration', 'msrp')

### Switching Axes ###

In [ ]:
correlation(t, 'x', 'y')

In [ ]:
t.scatter('x', 'y', s=30, color='red')

In [ ]:
t.scatter('y', 'x', s=30, color='red')

In [ ]:
correlation(t, 'y', 'x')

### Nonlinearity ###

In [ ]:
new_x = np.arange(-4, 4.1, 0.5)
nonlinear = Table().with_columns(
        'x', new_x,
        'y', new_x**2
    )
nonlinear.scatter('x', 'y', s=30, color='r')

In [ ]:
correlation(nonlinear, 'x', 'y')

### Outliers ###

In [ ]:
line = Table().with_columns(
        'x', make_array(1, 2, 3, 4),
        'y', make_array(1, 2, 3, 4)
    )
line.scatter('x', 'y', s=30, color='r')

In [ ]:
correlation(line, 'x', 'y')

In [ ]:
outlier = Table().with_columns(
        'x', make_array(1, 2, 3, 4, 5),
        'y', make_array(1, 2, 3, 4, 0)
    )
outlier.scatter('x', 'y', s=30, color='r')

In [ ]:
correlation(outlier, 'x', 'y')

### Ecological Correlations ###

In [ ]:
sat2014 = Table.read_table('data/sat2014.csv').sort('State')
sat2014

In [ ]:
sat2014.scatter('Critical Reading', 'Math')

In [ ]:
correlation(sat2014, 'Critical Reading', 'Math')